In [1]:
import pandas as pd
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
import os
import matplotlib.pyplot as plt
import seaborn as sns
import io
import base64
import json
import datetime 

In [ ]:
# --- Configuración de OpenAI ---
# Establece la API Key de OpenAI aquí
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')  

# Inicializa el modelo de embeddings y el LLM
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
llm = ChatOpenAI(model="gpt-4o", temperature=0.1) # gpt-4o o gpt-3.5-turbo se pueden usar aquí

In [3]:
# Declarar original_df como global para que pueda ser accesible en otras funciones
original_df = None 

In [ ]:
# --- Paso 1: Cargar y Procesar el Archivo CSV ---
def load_and_process_csv(file_path):
    """
    Función para cargar y procesar un archivo CSV, transformando cada fila en un objeto Document con metadatos.

    Parámetros:
    file_path (str): Ruta del archivo CSV a cargar.

    Retorna:
    list: Una lista de objetos Document, donde cada uno representa una fila del CSV con contenido y metadatos.
    """

    global original_df # Indicar que vamos a modificar la variable global
    try:
        df = pd.read_csv(file_path)
        original_df = df.copy() # Guardar una copia del DataFrame original globalmente
    except FileNotFoundError:
        print(f"Error: El archivo '{file_path}' no se encontró. Asegúrate de que está en la misma carpeta o la ruta es correcta.")
        return None

    documents = [] # Inicializa una lista vacía para almacenar los objetos Document
    for index, row in df.iterrows(): # Itera sobre cada fila del DataFrame
        # Convierte la fila del DataFrame en una cadena de texto
        content = ", ".join([f"{col}: {row[col]}" for col in df.columns])
        # Crea metadatos para el documento (en este caso, el índice de la fila original)
        metadata = {"row_index": index} 
        # Crea un objeto Document y lo añade a la lista 'documents'
        documents.append(Document(page_content=content, metadata=metadata))
    
    print(f"Se cargaron {len(documents)} documentos del archivo CSV.")
    return documents

In [ ]:
# --- Paso 2: Crear la Base de Datos Vectorial ---
def create_vector_db(documents, embeddings_model):
    """
    Función para crear una base de datos vectorial utilizando FAISS.

    Parámetros:
    documents (list): Lista de objetos Document que contienen el contenido y metadatos de las filas del CSV.
    embeddings_model: Modelo de embeddings para generar representaciones vectoriales de los documentos.

    Retorna:
    FAISS index: Base de datos vectorial generada a partir de los documentos.
    """
    print("Creando la base de datos vectorial... Esto puede tomar un momento.")
    # Usamos FAISS para una base de datos vectorial en memoria.
    vector_db = FAISS.from_documents(documents, embeddings_model)
    print("Base de datos vectorial creada exitosamente.")
    return vector_db

In [ ]:
# --- Paso 3: Crear el Agente de Consulta (RetrievalQA Chain) ---
def create_agent(vector_db, llm_model):
    """
    Función para configurar un agente de consulta basado en RetrievalQA, que combina recuperación de documentos con un modelo de lenguaje para responder preguntas.

    Parámetros:
    vector_db (FAISS index): Base de datos vectorial utilizada para almacenar y recuperar documentos relevantes.
    llm_model: Modelo de lenguaje grande (LLM) que se encargará de procesar las consultas y generar respuestas.

    Retorna:
    RetrievalQA: Cadena de recuperación y respuesta que permite consultar la base de datos vectorial con un LLM.
    """
    # Un "retriever" es lo que buscará documentos relevantes en tu base de datos vectorial.
    retriever = vector_db.as_retriever()
    
    # RetrievalQA es una cadena que combina la recuperación de documentos con un LLM para responder preguntas.
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm_model, 
        chain_type="stuff", # 'stuff' simplemente toma todos los documentos recuperados y los 'mete' en el prompt del LLM
        retriever=retriever, 
        return_source_documents=True # Útil para ver qué partes del CSV se usaron para la respuesta
    )
    print("Agente de consulta configurado.")
    return qa_chain

In [ ]:
# --- Función para generar la gráfica  ---
def create_plot(df_data, plot_type='line', x_col='Fecha', y_col='Ventas', hue_col=None, title='Ventas a lo Largo del Tiempo'):
    """
    Función para crear y guardar una gráfica basada en los datos proporcionados.

    Parámetros:
    df_data (DataFrame): DataFrame con los datos a visualizar.
    plot_type (str): Tipo de gráfica a generar ('line' para línea, 'bar' para barras).
    x_col (str): Nombre de la columna que se usará en el eje X.
    y_col (str): Nombre de la columna que se usará en el eje Y.
    hue_col (str, opcional): Columna para diferenciar datos por colores.
    title (str): Título de la gráfica.

    Retorna:
    str: Ruta del archivo donde se guardó la gráfica o None si hubo un error.
    """

    plt.figure(figsize=(10, 6))
    
    if 'Fecha' in df_data.columns:
        df_data['Fecha'] = pd.to_datetime(df_data['Fecha'])
    
    if plot_type == 'line':
        sns.lineplot(data=df_data, x=x_col, y=y_col, hue=hue_col)
    elif plot_type == 'bar':
        if hue_col:
            sns.barplot(data=df_data, x=x_col, y=y_col, hue=hue_col, estimator='sum', errorbar=None)
        else:
            sns.barplot(data=df_data, x=x_col, y=y_col, estimator='sum', errorbar=None)
    else:
        print(f"Tipo de gráfica '{plot_type}' no soportado aún.")
        plt.close()
        return None

    plt.title(title)
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    # --- Guardar la gráfica en un archivo ---
    # Generar un nombre de archivo único basado en la marca de tiempo y el título
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    # Limpiar el título para usarlo en el nombre del archivo (quitar caracteres especiales)
    safe_title = "".join(c for c in title if c.isalnum() or c in (' ', '_')).replace(' ', '_')[:50] # Limitar longitud
    file_name = f"grafica_{safe_title}_{timestamp}.png"
    file_path = os.path.join(os.getcwd(), file_name) # Guarda en la carpeta actual del proyecto

    try:
        plt.savefig(file_path, format='png', bbox_inches='tight')
        print(f"\n--- Gráfica guardada como: {file_name} ---")
    except Exception as e:
        print(f"Error al guardar la gráfica como archivo: {e}")
        file_path = None # Indica que no se pudo guardar

    plt.close() # Cierra la figura de Matplotlib
    # Retorna la ruta del archivo guardado
    return file_path # Retorna la ruta del archivo si se guardó, o None

In [ ]:
# --- Función para extraer la intención de visualización usando el LLM ---
def extract_plot_intent(query, retrieved_documents):
    """
    Función para extraer la intención de visualización de datos a partir de una consulta del usuario 
    y documentos relevantes recuperados.

    Parámetros:
    query (str): Consulta del usuario que puede contener una solicitud de visualización de datos.
    retrieved_documents (list): Lista de objetos Document recuperados que contienen información relevante.

    Retorna:
    dict: Un diccionario con los parámetros de visualización si la consulta solicita una gráfica, 
          o {"is_plot_request": False} si no se identifica una intención de graficar.
    """
    # Crear un contexto a partir de los documentos recuperados
    context = "\n".join([doc.page_content for doc in retrieved_documents])

    # Construir un prompt para el LLM para extraer la intención de visualización
    # Le pedimos al LLM que nos devuelva un formato JSON para que sea fácil de parsear.
    prompt_for_plot_intent = f"""
    Basado en la siguiente consulta del usuario y los datos relevantes extraídos de un archivo CSV:

    Consulta del usuario: "{query}"

    Datos relevantes (fila del CSV):
    {context}

    Analiza la consulta y los datos. Si la consulta parece pedir una visualización de datos (ej. "graficar", "mostrar un gráfico", "visualizar ventas", "comparar ventas por país"), extrae la siguiente información en formato JSON. Si no parece una solicitud de visualización, devuelve un JSON vacío o nulo.

    Formato JSON esperado:
    {{
        "is_plot_request": true/false,
        "plot_type": "line" o "bar" (basado en la consulta si es posible, por defecto "line"),
        "x_axis": "Fecha" (por defecto o inferido),
        "y_axis": "Ventas" (por defecto o inferido),
        "group_by": "Country" o "Coffee type" o null (si se pide diferenciar por algo),
        "title": "Título sugerido para la gráfica"
    }}

    Considera las columnas disponibles: 'Fecha', 'Country', 'Ventas', 'Coffee type'.
    Si el usuario no especifica, asume 'Fecha' para x_axis y 'Ventas' para y_axis.
    Si no se puede inferir 'plot_type', asume 'line'.

    Ejemplos de consultas que implican gráfica:
    - "Muéstrame las ventas de Angola a lo largo del tiempo." -> {{ "is_plot_request": true, "plot_type": "line", "x_axis": "Fecha", "y_axis": "Ventas", "group_by": null, "title": "Ventas de Angola a lo largo del tiempo" }}
    - "Grafica las ventas por país." -> {{ "is_plot_request": true, "plot_type": "bar", "x_axis": "Country", "y_axis": "Ventas", "group_by": null, "title": "Ventas totales por país" }}
    - "Quiero ver las ventas de café por tipo." -> {{ "is_plot_request": true, "plot_type": "bar", "x_axis": "Coffee type", "y_axis": "Ventas", "group_by": null, "title": "Ventas por tipo de café" }}
    - "Compara las ventas por país a lo largo de los años." -> {{ "is_plot_request": true, "plot_type": "line", "x_axis": "Fecha", "y_axis": "Ventas", "group_by": "Country", "title": "Comparación de Ventas por País a lo largo de los Años" }}
    - "Cuál fue la venta más alta?" -> {{ "is_plot_request": false }}

    Respuesta JSON:
    """
    
    try:
        # Usar el LLM para extraer la intención
        response = llm.invoke(prompt_for_plot_intent)
        json_output = response.content.strip()

        # A veces el LLM puede añadir algo de texto antes o después del JSON
        # Intentamos limpiar la respuesta para asegurarnos de que sea un JSON válido
        if json_output.startswith("```json"):
            json_output = json_output[7:]
        if json_output.endswith("```"):
            json_output = json_output[:-3]
        
        plot_params = json.loads(json_output)
        return plot_params
    except json.JSONDecodeError as e:
        print(f"Advertencia: No se pudo parsear la respuesta del LLM como JSON. Error: {e}")
        print(f"Respuesta cruda del LLM: {response.content}")
        return {"is_plot_request": False}
    except Exception as e:
        print(f"Ocurrió un error al extraer la intención de la gráfica: {e}")
        return {"is_plot_request": False}


In [ ]:
# --- Función Principal ---
def main():
    """
    Función principal del sistema de consulta y generación de gráficos.

    Flujo general:
    1. Carga y procesamiento del archivo CSV.
    2. Creación de la base de datos vectorial con embeddings.
    3. Configuración del agente de consulta basado en RetrievalQA.
    4. Interacción con el usuario para hacer consultas y generar gráficas según la intención detectada.

    Retorna:
    None: La función es interactiva y no devuelve valores.
    """
    csv_file_path = "coffee_final.csv" 

    documents = load_and_process_csv(csv_file_path)
    if not documents:
        return

    vector_db = create_vector_db(documents, embeddings)
    qa_agent = create_agent(vector_db, llm)

    print("\n¡Listo para hacer consultas! Escribe 'salir' para terminar.")
    print("Intenta pedir gráficas, por ejemplo: 'Grafica las ventas por país' o 'Muéstrame las ventas de Angola a lo largo del tiempo'.")
    
    while True:
        query = input("\nTu pregunta: ")
        if query.lower() == 'salir':
            print("¡Hasta luego!")
            break
        
        try:
            # Primero, obtenemos la respuesta normal del agente y los documentos fuente
            result = qa_agent.invoke({"query": query})
            
            print("\n--- Respuesta del Agente ---")
            print(result["result"])
            
            print("\n--- Documentos Fuente Utilizados ---")
            for doc in result["source_documents"]:
                print(f"- {doc.page_content}")
            print("-----------------------------\n")

            # --- Lógica de Visualización ---
            # 1. Intentar extraer la intención de la gráfica
            plot_intent = extract_plot_intent(query, result["source_documents"])

            if plot_intent.get("is_plot_request"):
                print("Detectada intención de gráfica. Intentando generar visualización...")
                
                plot_type = plot_intent.get("plot_type", "line")
                x_axis = plot_intent.get("x_axis", "Fecha")
                y_axis = plot_intent.get("y_axis", "Ventas")
                group_by = plot_intent.get("group_by")
                title = plot_intent.get("title", f"Gráfica de {y_axis} vs {x_axis}")

                # Filtrar el DataFrame original si la consulta implica un país o tipo de café específico
                # Esto es una simplificación y puede necesitar una lógica más robusta
                df_for_plot = original_df.copy() # Usamos una copia del DataFrame global
                
                # Intentamos inferir filtros basados en la consulta y los documentos relevantes
                # Esto podría mejorarse con un LLM para extraer filtros explícitamente
                query_lower = query.lower()
                
                # Asegurarse de que las columnas existen en el DataFrame filtrado
                required_cols = [x_axis, y_axis]
                if group_by:
                    required_cols.append(group_by)
                
                if all(col in df_for_plot.columns for col in required_cols):
                    # Generar la gráfica
                    plot_image_bytes = create_plot(
                        df_for_plot, 
                        plot_type=plot_type, 
                        x_col=x_axis, 
                        y_col=y_axis, 
                        hue_col=group_by, 
                        title=title
                    )
                    
                    if plot_image_bytes:
                        # Si estás en un entorno como Jupyter/Colab, podrías mostrar la imagen directamente
                        # from IPython.display import Image, display
                        # display(Image(plot_image_bytes))
                        print("Gráfica generada y lista para mostrar.")
                    else:
                        print("No se pudo generar la gráfica.")
                else:
                    print(f"Advertencia: No se encontraron las columnas necesarias para graficar ({required_cols}).")
            else:
                print("No se detectó una solicitud de gráfica en la consulta.")

        except Exception as e:
            print(f"Ocurrió un error al procesar la consulta: {e}")


In [12]:
if __name__ == "__main__":
    main()

Se cargaron 1430 documentos del archivo CSV.
Creando la base de datos vectorial... Esto puede tomar un momento.
Base de datos vectorial creada exitosamente.
Agente de consulta configurado.

¡Listo para hacer consultas! Escribe 'salir' para terminar.
Intenta pedir gráficas, por ejemplo: 'Grafica las ventas por país' o 'Muéstrame las ventas de Angola a lo largo del tiempo'.

--- Respuesta del Agente ---
Lo siento, pero no tengo información sobre las ventas de café de Colombia. Solo tengo datos de ventas de Ecuador para los años 2005, 2013, 2014 y 2015.

--- Documentos Fuente Utilizados ---
- Fecha: 2013-01-01, Country: Ecuador, Ventas: 9000000, Coffee type: Arabica/Robusta
- Fecha: 2005-01-01, Country: Ecuador, Ventas: 9000000, Coffee type: Arabica/Robusta
- Fecha: 2015-01-01, Country: Ecuador, Ventas: 9300000, Coffee type: Arabica/Robusta
- Fecha: 2014-01-01, Country: Ecuador, Ventas: 9300000, Coffee type: Arabica/Robusta
-----------------------------

Detectada intención de gráfica. In

/tmp/ipykernel_6101/3833760694.py:24: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()



--- Gráfica guardada como: grafica_Ventas_anuales_de_Colombia_y_Ecuador_20250608_110835.png ---
Gráfica generada y lista para mostrar.
¡Hasta luego!


In [ ]:
#!jupyter nbconvert --to script NB_agente_consulta_coffe.ipynb